In [508]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import linregress
import ast

I am going to flesh out the features for the 'matches' table. I will start off by selecting just a couple of obvious ones, and then I will go back over things and get more features. 

I will start by looking at the "events" table and aggregating by the matchId identifier and joining these juicy stats onto the match table.

In [509]:
events = pd.read_csv('data/events_England.csv')

## About this file

This dataset describes all the events that occur during each match. Each event refers to a ball touch and contains the following information:

- **eventId**: the identifier of the event's type. Each eventId is associated with an event name.
- **eventName**: the name of the event's type (pass, foul, shot, duel, free kick, offside, or touch).
- **subEventId**: the identifier of the subevent's type.
- **subEventName**: the name of the subevent's type, associated with different event types.
- **tags**: a list of event tags providing additional information about the event (e.g., accurate).
- **eventSec**: the time the event occurs (in seconds since the beginning of the current half).
- **id**: a unique identifier of the event.
- **matchId**: the identifier of the match, linked to "wyId" in the match dataset.
- **matchPeriod**: the period of the match ("1H", "2H", "E1", "E2", or "P").
- **playerId**: the identifier of the player who generated the event, linked to "wyId" in a player dataset.
- **positions**: the origin and destination positions associated with the event as (x, y) coordinates.
- **teamId**: the identifier of the player's team, linked to "wyId" in the team dataset.
- **tagsList**: same information as the tags column, except in list format.
- **pos_orig_x**: from the positions column, origin in x-axis.
- **pos_orig_y**: from the positions column, origin in y-axis.
- **pos_dest_x**: from the positions column, destination in x-axis.
- **pos_dest_y**: from the positions column, destination in y-axis.


In [510]:
events.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,tagsList,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85.0,177959171,[1801],49,49,78,31
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83.0,177959172,[1801],78,31,75,51
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82.0,177959173,[1801],75,51,71,35
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82.0,177959174,[1801],71,35,95,41
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85.0,177959175,[1801],95,41,88,72


### teams

I am updating the teamsid to be the team name already because it helps me confirm the data is correct

In [511]:
teams=pd.read_csv('data/teams.csv')
teams.head()


,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alav\u00e9s,696,Deportivo Alav\u00e9s,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [512]:
team_mapping=teams.set_index('wyId')['name'].to_dict()

In [513]:
events['teamId']=events['teamId'].map(team_mapping)


In [514]:
events.rename(columns={'teamId':'teamName'}, inplace=True)

### tags2name

In [515]:
tags2name = pd.read_csv('data/tags2name.csv')

##### Joining on the tag labels to the nested tagsList column

In [516]:
%%time
##takes about 42 seconds to run

#getting the list values out of the string values in the tagList column
events['tagsList'] = events['tagsList'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

#explode the tagsList column because we will get the labels
exploded_events = events.explode('tagsList')

#cast to int because the column in the tags table is int
exploded_events['tagsList']=exploded_events['tagsList'].astype('Int64')

#keep the original index because I was losing this information in the merge 
exploded_events['origIndex']=exploded_events.index

#get the tag labels
exploded_merged_events=exploded_events.merge(tags2name[['Tag','Label']], left_on='tagsList', right_on='Tag', how='left')

#grouping by original index to prepare for the join back to events table
exploded_merged_events=exploded_merged_events.groupby('origIndex').agg({
   # 'tagsList': list, 
   # 'Tag': list,
    'Label': list
}).reset_index()

#joining back onto original events table
events=pd.concat([events, exploded_merged_events['Label']], axis=1)

CPU times: user 25.4 s, sys: 1.05 s, total: 26.5 s
Wall time: 26.5 s


##### Removing some columns that are no longer needed

In [517]:
events.drop(['eventId','subEventId','tagsList','positions','tags','id'], axis=1, inplace=True)

In [518]:
events.head(8)

,subEventName,playerId,matchId,eventName,teamName,matchPeriod,eventSec,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x,Label
0,Simple pass,25413,2499719,Pass,Arsenal,1H,2.758649,49,49,78,31,[accurate]
1,High pass,370224,2499719,Pass,Arsenal,1H,4.946850,78,31,75,51,[accurate]
2,Head pass,3319,2499719,Pass,Arsenal,1H,6.542188,75,51,71,35,[accurate]
3,Head pass,120339,2499719,Pass,Arsenal,1H,8.143395,71,35,95,41,[accurate]
4,Simple pass,167145,2499719,Pass,Arsenal,1H,10.302366,95,41,88,72,[accurate]
5,Simple pass,3319,2499719,Pass,Arsenal,1H,12.548934,88,72,75,77,[not accurate]
6,Head pass,8653,2499719,Pass,Leicester City,1H,13.961228,25,23,15,39,[accurate]
7,Air duel,8013,2499719,Duel,Leicester City,1H,14.765321,15,39,20,33,"[lost, not accurate]"


Number of shots in each game

In [519]:
events.groupby(['matchId','teamName']).agg(
    numShots=('subEventName', lambda x: (x == 'Shot').sum())
)

##I can join this onto the matches table later

numShots
matchId teamName                        
2499719 Arsenal                       27
        Leicester City                 7
2499720 Brighton & Hove Albion         6
        Manchester City               12
2499721 Burnley                        9
...                                  ...
2500096 Swansea City                  24
2500097 Leicester City                17
        Tottenham Hotspur             13
2500098 Everton                       13
        West Ham United               14

[760 rows x 1 columns]

## Matches

In [520]:
matches = pd.read_csv('data/matches_England.csv')

In [521]:
##Dropping unneeded columns columns are all either identical or they are captured within other columns

In [522]:
matches.drop(['status','roundId','seasonId','duration','competitionId','teamsData','team1.formation','team2.formation','team1.scoreET','team1.scoreP','team1.hasFormation','team2.scoreET','team2.scoreP','team2.hasFormation','date'], axis=1, inplace=True)

In [523]:
##And then dropping the subs bench info

In [524]:
matches.drop(['team1.formation.bench','team2.formation.bench'], axis=1, inplace=True)

In [525]:
#dropping venue too, but may use it at some stage on dashboards

In [526]:
matches.drop(['venue'], axis=1, inplace=True)

In [527]:
#dropping venue too, but may use it at some stage for features or for dashboards

In [528]:
matches.drop(['referees'], axis=1, inplace=True)

I want to rejig my table so that team1 columns always refer to the home team, and team2 columns refer to the away team.

first ill replace the teamId's with the actual team names so that I know this is working as expected

### Teams

In [529]:
teams=pd.read_csv('data/teams.csv')
teams.head()
team_mapping=teams.set_index('wyId')['name'].to_dict()

In [530]:
matches['team1.teamId']=matches['team1.teamId'].map(team_mapping)
matches['team2.teamId']=matches['team2.teamId'].map(team_mapping)
matches['winner']=matches['winner'].map(team_mapping)

In [531]:
matches.rename(columns={'team1.teamId':'team1.teamName','team2.teamId':'team2.teamName'}, inplace=True)

In [532]:
matches

,gameweek,dateutc,winner,wyId,label,team1.coachId,team1.side,team1.teamName,team1.score,team1.scoreHT,team1.formation.lineup,team1.formation.substitutions,team2.coachId,team2.side,team2.teamName,team2.score,team2.scoreHT,team2.formation.lineup,team2.formation.substitutions
0,38,2018-05-13 14:00:00,AFC Bournemouth,2500089,"Burnley - AFC Bournemouth, 1 - 2",8880,home,Burnley,1,1,"[{'playerId': 9206, 'ownGoals': '0', 'redCards...","[{'playerIn': 9127, 'playerOut': 9206, 'minute...",8934,away,AFC Bournemouth,2,0,"[{'playerId': 259531, 'ownGoals': '0', 'redCar...","[{'playerIn': 7989, 'playerOut': 259531, 'minu..."
1,38,2018-05-13 14:00:00,Crystal Palace,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0",8357,home,Crystal Palace,2,0,"[{'playerId': 8623, 'ownGoals': '0', 'redCards...","[{'playerIn': 38031, 'playerOut': 8623, 'minut...",0,away,West Bromwich Albion,0,0,"[{'playerId': 25537, 'ownGoals': '0', 'redCard...","[{'playerIn': 261, 'playerOut': 25537, 'minute..."
2,38,2018-05-13 14:00:00,Arsenal,2500091,"Huddersfield Town - Arsenal, 0 - 1",7845,away,Arsenal,1,1,"[{'playerId': 25867, 'ownGoals': '0', 'redCard...","[{'playerIn': 7945, 'playerOut': 25867, 'minut...",18572,home,Huddersfield Town,0,0,"[{'playerId': 9419, 'ownGoals': '0', 'redCards...","[{'playerIn': 38377, 'playerOut': 9419, 'minut..."
3,38,2018-05-13 14:00:00,Liverpool,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0",8093,away,Brighton & Hove Albion,0,0,"[{'playerId': 120, 'ownGoals': '2', 'redCards'...","[{'playerIn': 15526, 'playerOut': 120, 'minute...",14791,home,Liverpool,4,2,"[{'playerId': 25747, 'ownGoals': '2', 'redCard...","[{'playerIn': 8140, 'playerOut': 25747, 'minut..."
4,38,2018-05-13 14:00:00,Manchester United,2500093,"Manchester United - Watford, 1 - 0",93112,away,Watford,0,0,"[{'playerId': 68085, 'ownGoals': '0', 'redCard...","[{'playerIn': 8889, 'playerOut': 68085, 'minut...",3295,home,Manchester United,1,1,"[{'playerId': 7939, 'ownGoals': '0', 'redCards...","[{'playerIn': 8135, 'playerOut': 7939, 'minute..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1,2017-08-12 14:00:00,Everton,2499723,"Everton - Stoke City, 1 - 0",9054,away,Stoke City,0,0,"[{'playerId': 20450, 'ownGoals': '0', 'redCard...","[{'playerIn': 15198, 'playerOut': 20450, 'minu...",268779,home,Everton,1,1,"[{'playerId': 77546, 'ownGoals': '0', 'redCard...","[{'playerIn': 340, 'playerOut': 77546, 'minute..."
376,1,2017-08-12 14:00:00,NaN,2499726,"Southampton - Swansea City, 0 - 0",254174,home,Southampton,0,0,"[{'playerId': 20857, 'ownGoals': '0', 'redCard...","[{'playerIn': 8953, 'playerOut': 20857, 'minut...",381291,away,Swansea City,0,0,"[{'playerId': 77550, 'ownGoals': '0', 'redCard...","[{'playerIn': 62344, 'playerOut': 77550, 'minu..."
377,1,2017-08-12 14:00:00,West Bromwich Albion,2499728,"West Bromwich Albion - AFC Bournemouth, 1 - 0",8934,away,AFC Bournemouth,0,0,"[{'playerId': 9293, 'ownGoals': '0', 'redCards...","[{'playerIn': 9739, 'playerOut': 9293, 'minute...",149026,home,West Bromwich Albion,1,1,"[{'playerId': 434159, 'ownGoals': '0', 'redCar...","[{'playerIn': 447254, 'playerOut': 434159, 'mi..."
378,1,2017-08-12 11:30:00,NaN,2499727,"Watford - Liverpool, 3 - 3",14791,away,Liverpool,3,1,"[{'playerId': 15808, 'ownGoals': '0', 'redCard...","[{'playerIn': 134708, 'playerOut': 15808, 'min...",71037,home,Watford,3,2,"[{'playerId': 160, 'ownGoals': '0', 'redCards'...","[{'playerIn': 3351, 'playerOut': 160, 'minute'..."


#### I want all home teams on the left, all the away teams on the right. And I want it labeled home and away instead of team1 and team2

In [533]:
home_teams1 = matches[matches['team1.side'] == 'home'].copy()

In [534]:
home_teams1=home_teams1[['team1.coachId',
       'team1.side', 'team1.teamName', 'team1.score', 'team1.scoreHT',
       'team1.formation.lineup', 'team1.formation.substitutions']]

In [535]:
home_teams1.columns=['home.coachId', 'home.side', 'home.teamName', 'home.score',
       'home.scoreHT', 'home.formation.lineup',
       'home.formation.substitutions']

In [536]:
home_teams2 = matches[matches['team2.side'] == 'home'].copy()

In [537]:
home_teams2=home_teams2[['team2.coachId', 'team2.side', 'team2.teamName', 'team2.score',
       'team2.scoreHT', 'team2.formation.lineup',
       'team2.formation.substitutions']]

In [538]:
home_teams2.columns=['home.coachId', 'home.side', 'home.teamName', 'home.score',
       'home.scoreHT', 'home.formation.lineup',
       'home.formation.substitutions']

In [539]:

home_teams=pd.concat([home_teams1, home_teams2], axis=0).sort_index()

Doing the same for the away teams

In [540]:
away_teams1 = matches[matches['team1.side'] == 'away'].copy()

away_teams1=away_teams1[['team1.coachId',
       'team1.side', 'team1.teamName', 'team1.score', 'team1.scoreHT',
       'team1.formation.lineup', 'team1.formation.substitutions']]

away_teams1.columns=['away.coachId', 'away.side', 'away.teamName', 'away.score',
       'away.scoreHT', 'away.formation.lineup',
       'away.formation.substitutions']



away_teams2 = matches[matches['team2.side'] == 'away'].copy()

away_teams2=away_teams2[['team2.coachId', 'team2.side', 'team2.teamName', 'team2.score',
       'team2.scoreHT', 'team2.formation.lineup',
       'team2.formation.substitutions']]

away_teams2.columns=['away.coachId', 'away.side', 'away.teamName', 'away.score',
       'away.scoreHT', 'away.formation.lineup',
       'away.formation.substitutions']

away_teams=pd.concat([away_teams1, away_teams2], axis=0).sort_index()

joining all this back onto the matches table

In [541]:
matches[['gameweek', 'dateutc', 'winner', 'wyId', 'label']]

,gameweek,dateutc,winner,wyId,label
0,38,2018-05-13 14:00:00,AFC Bournemouth,2500089,"Burnley - AFC Bournemouth, 1 - 2"
1,38,2018-05-13 14:00:00,Crystal Palace,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0"
2,38,2018-05-13 14:00:00,Arsenal,2500091,"Huddersfield Town - Arsenal, 0 - 1"
3,38,2018-05-13 14:00:00,Liverpool,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0"
4,38,2018-05-13 14:00:00,Manchester United,2500093,"Manchester United - Watford, 1 - 0"
...,...,...,...,...,...
375,1,2017-08-12 14:00:00,Everton,2499723,"Everton - Stoke City, 1 - 0"
376,1,2017-08-12 14:00:00,NaN,2499726,"Southampton - Swansea City, 0 - 0"
377,1,2017-08-12 14:00:00,West Bromwich Albion,2499728,"West Bromwich Albion - AFC Bournemouth, 1 - 0"
378,1,2017-08-12 11:30:00,NaN,2499727,"Watford - Liverpool, 3 - 3"


In [542]:
match_cols=matches[['gameweek', 'dateutc', 'winner', 'wyId', 'label']].copy()

In [543]:
matches=pd.concat([match_cols , home_teams, away_teams], axis = 1)

In [544]:
matches

,gameweek,dateutc,winner,wyId,label,home.coachId,home.side,home.teamName,home.score,home.scoreHT,home.formation.lineup,home.formation.substitutions,away.coachId,away.side,away.teamName,away.score,away.scoreHT,away.formation.lineup,away.formation.substitutions
0,38,2018-05-13 14:00:00,AFC Bournemouth,2500089,"Burnley - AFC Bournemouth, 1 - 2",8880,home,Burnley,1,1,"[{'playerId': 9206, 'ownGoals': '0', 'redCards...","[{'playerIn': 9127, 'playerOut': 9206, 'minute...",8934,away,AFC Bournemouth,2,0,"[{'playerId': 259531, 'ownGoals': '0', 'redCar...","[{'playerIn': 7989, 'playerOut': 259531, 'minu..."
1,38,2018-05-13 14:00:00,Crystal Palace,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0",8357,home,Crystal Palace,2,0,"[{'playerId': 8623, 'ownGoals': '0', 'redCards...","[{'playerIn': 38031, 'playerOut': 8623, 'minut...",0,away,West Bromwich Albion,0,0,"[{'playerId': 25537, 'ownGoals': '0', 'redCard...","[{'playerIn': 261, 'playerOut': 25537, 'minute..."
2,38,2018-05-13 14:00:00,Arsenal,2500091,"Huddersfield Town - Arsenal, 0 - 1",18572,home,Huddersfield Town,0,0,"[{'playerId': 9419, 'ownGoals': '0', 'redCards...","[{'playerIn': 38377, 'playerOut': 9419, 'minut...",7845,away,Arsenal,1,1,"[{'playerId': 25867, 'ownGoals': '0', 'redCard...","[{'playerIn': 7945, 'playerOut': 25867, 'minut..."
3,38,2018-05-13 14:00:00,Liverpool,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0",14791,home,Liverpool,4,2,"[{'playerId': 25747, 'ownGoals': '2', 'redCard...","[{'playerIn': 8140, 'playerOut': 25747, 'minut...",8093,away,Brighton & Hove Albion,0,0,"[{'playerId': 120, 'ownGoals': '2', 'redCards'...","[{'playerIn': 15526, 'playerOut': 120, 'minute..."
4,38,2018-05-13 14:00:00,Manchester United,2500093,"Manchester United - Watford, 1 - 0",3295,home,Manchester United,1,1,"[{'playerId': 7939, 'ownGoals': '0', 'redCards...","[{'playerIn': 8135, 'playerOut': 7939, 'minute...",93112,away,Watford,0,0,"[{'playerId': 68085, 'ownGoals': '0', 'redCard...","[{'playerIn': 8889, 'playerOut': 68085, 'minut..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1,2017-08-12 14:00:00,Everton,2499723,"Everton - Stoke City, 1 - 0",268779,home,Everton,1,1,"[{'playerId': 77546, 'ownGoals': '0', 'redCard...","[{'playerIn': 340, 'playerOut': 77546, 'minute...",9054,away,Stoke City,0,0,"[{'playerId': 20450, 'ownGoals': '0', 'redCard...","[{'playerIn': 15198, 'playerOut': 20450, 'minu..."
376,1,2017-08-12 14:00:00,NaN,2499726,"Southampton - Swansea City, 0 - 0",254174,home,Southampton,0,0,"[{'playerId': 20857, 'ownGoals': '0', 'redCard...","[{'playerIn': 8953, 'playerOut': 20857, 'minut...",381291,away,Swansea City,0,0,"[{'playerId': 77550, 'ownGoals': '0', 'redCard...","[{'playerIn': 62344, 'playerOut': 77550, 'minu..."
377,1,2017-08-12 14:00:00,West Bromwich Albion,2499728,"West Bromwich Albion - AFC Bournemouth, 1 - 0",149026,home,West Bromwich Albion,1,1,"[{'playerId': 434159, 'ownGoals': '0', 'redCar...","[{'playerIn': 447254, 'playerOut': 434159, 'mi...",8934,away,AFC Bournemouth,0,0,"[{'playerId': 9293, 'ownGoals': '0', 'redCards...","[{'playerIn': 9739, 'playerOut': 9293, 'minute..."
378,1,2017-08-12 11:30:00,NaN,2499727,"Watford - Liverpool, 3 - 3",71037,home,Watford,3,2,"[{'playerId': 160, 'ownGoals': '0', 'redCards'...","[{'playerIn': 3351, 'playerOut': 160, 'minute'...",14791,away,Liverpool,3,1,"[{'playerId': 15808, 'ownGoals': '0', 'redCard...","[{'playerIn': 134708, 'playerOut': 15808, 'min..."


Now I will join on the data I grouped for earlier

In [545]:
numshots=events.groupby(['matchId','teamName']).agg(
    numShots=('subEventName', lambda x: (x == 'Shot').sum())
).reset_index()

In [546]:
numshots

,matchId,teamName,numShots
0,2499719,Arsenal,27
1,2499719,Leicester City,7
2,2499720,Brighton & Hove Albion,6
3,2499720,Manchester City,12
4,2499721,Burnley,9
...,...,...,...
755,2500096,Swansea City,24
756,2500097,Leicester City,17
757,2500097,Tottenham Hotspur,13
758,2500098,Everton,13


In [547]:
matches=matches.merge(
    numshots, 
    how='left', 
    left_on=['wyId','home.teamName'], 
    right_on=['matchId','teamName']
).rename(columns={'numShots':'home.numShots'})

matches.drop(['matchId','teamName'], axis=1, inplace=True)

In [548]:
matches=matches.merge(
    numshots, 
    how='left', 
    left_on=['wyId','away.teamName'], 
    right_on=['matchId','teamName']
).rename(columns={'numShots':'away.numShots'})

matches.drop(['matchId','teamName'], axis=1, inplace=True)

In [549]:
#numshots[numshots['matchId'].isin([2500089,2500090])]

In [550]:
matches.head()

,gameweek,dateutc,winner,wyId,label,home.coachId,home.side,home.teamName,home.score,home.scoreHT,...,home.formation.substitutions,away.coachId,away.side,away.teamName,away.score,away.scoreHT,away.formation.lineup,away.formation.substitutions,home.numShots,away.numShots
0,38,2018-05-13 14:00:00,AFC Bournemouth,2500089,"Burnley - AFC Bournemouth, 1 - 2",8880,home,Burnley,1,1,...,"[{'playerIn': 9127, 'playerOut': 9206, 'minute...",8934,away,AFC Bournemouth,2,0,"[{'playerId': 259531, 'ownGoals': '0', 'redCar...","[{'playerIn': 7989, 'playerOut': 259531, 'minu...",9,14
1,38,2018-05-13 14:00:00,Crystal Palace,2500090,"Crystal Palace - West Bromwich Albion, 2 - 0",8357,home,Crystal Palace,2,0,...,"[{'playerIn': 38031, 'playerOut': 8623, 'minut...",0,away,West Bromwich Albion,0,0,"[{'playerId': 25537, 'ownGoals': '0', 'redCard...","[{'playerIn': 261, 'playerOut': 25537, 'minute...",11,5
2,38,2018-05-13 14:00:00,Arsenal,2500091,"Huddersfield Town - Arsenal, 0 - 1",18572,home,Huddersfield Town,0,0,...,"[{'playerIn': 38377, 'playerOut': 9419, 'minut...",7845,away,Arsenal,1,1,"[{'playerId': 25867, 'ownGoals': '0', 'redCard...","[{'playerIn': 7945, 'playerOut': 25867, 'minut...",18,8
3,38,2018-05-13 14:00:00,Liverpool,2500092,"Liverpool - Brighton & Hove Albion, 4 - 0",14791,home,Liverpool,4,2,...,"[{'playerIn': 8140, 'playerOut': 25747, 'minut...",8093,away,Brighton & Hove Albion,0,0,"[{'playerId': 120, 'ownGoals': '2', 'redCards'...","[{'playerIn': 15526, 'playerOut': 120, 'minute...",21,2
4,38,2018-05-13 14:00:00,Manchester United,2500093,"Manchester United - Watford, 1 - 0",3295,home,Manchester United,1,1,...,"[{'playerIn': 8135, 'playerOut': 7939, 'minute...",93112,away,Watford,0,0,"[{'playerId': 68085, 'ownGoals': '0', 'redCard...","[{'playerIn': 8889, 'playerOut': 68085, 'minut...",7,5


Dropping these columns for now. I will focus on adding features at the team and match level. I will add player level information later.

In [551]:
matches.drop(['home.formation.lineup', 'home.formation.substitutions','away.formation.lineup', 'away.formation.substitutions'],
             axis=1,
             inplace=True
            )


In [552]:
matches.drop(['home.coachId','away.coachId'],
             axis=1,
             inplace=True
            )


sorting matches by date

In [553]:
matches=matches.sort_values(by='dateutc').reset_index(drop=True)

In [554]:
matches.head()

,gameweek,dateutc,winner,wyId,label,home.side,home.teamName,home.score,home.scoreHT,away.side,away.teamName,away.score,away.scoreHT,home.numShots,away.numShots
0,1,2017-08-11 18:45:00,Arsenal,2499719,"Arsenal - Leicester City, 4 - 3",home,Arsenal,4,2,away,Leicester City,3,2,27,7
1,1,2017-08-12 11:30:00,NaN,2499727,"Watford - Liverpool, 3 - 3",home,Watford,3,2,away,Liverpool,3,1,8,12
2,1,2017-08-12 14:00:00,NaN,2499726,"Southampton - Swansea City, 0 - 0",home,Southampton,0,0,away,Swansea City,0,0,25,4
3,1,2017-08-12 14:00:00,Burnley,2499721,"Chelsea - Burnley, 2 - 3",home,Chelsea,2,0,away,Burnley,3,3,15,9
4,1,2017-08-12 14:00:00,West Bromwich Albion,2499728,"West Bromwich Albion - AFC Bournemouth, 1 - 0",home,West Bromwich Albion,1,1,away,AFC Bournemouth,0,0,16,8


With the information I have now I might try build a basic ML model.

* I can  measure what team is home or away
* I can measure average points they have gained in the last 5 weeks
* average points they have gained all season
* their average shots in the last 5 games
* how many goals they have scored or conceded in the last 5 games.

This is all a drop in the ocean to the amount of data I can get. But I will start from here.

## measuring average points

I believe the best way to get the average points is to get the running total from the events table (I just need to sum up the goals for each game to determin a winner, which should be straightforward)

In [555]:
events.head()

,subEventName,playerId,matchId,eventName,teamName,matchPeriod,eventSec,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x,Label
0,Simple pass,25413,2499719,Pass,Arsenal,1H,2.758649,49,49,78,31,[accurate]
1,High pass,370224,2499719,Pass,Arsenal,1H,4.946850,78,31,75,51,[accurate]
2,Head pass,3319,2499719,Pass,Arsenal,1H,6.542188,75,51,71,35,[accurate]
3,Head pass,120339,2499719,Pass,Arsenal,1H,8.143395,71,35,95,41,[accurate]
4,Simple pass,167145,2499719,Pass,Arsenal,1H,10.302366,95,41,88,72,[accurate]


In [556]:
events.groupby(['matchId','teamName']).agg(
    numShots=('subEventName', lambda x: (x == 'Shot').sum())
).reset_index()

,matchId,teamName,numShots
0,2499719,Arsenal,27
1,2499719,Leicester City,7
2,2499720,Brighton & Hove Albion,6
3,2499720,Manchester City,12
4,2499721,Burnley,9
...,...,...,...
755,2500096,Swansea City,24
756,2500097,Leicester City,17
757,2500097,Tottenham Hotspur,13
758,2500098,Everton,13


I need to get all the rows that equate to a goal. and then aggregate by match and team.

In [557]:
events[events['Label'].apply(lambda x: ('Goal' in x) | ('own_goal' in x))]['eventName'].value_counts()

eventName
Save attempt          1019
Shot                   914
Free Kick               74
Others on the ball      27
Pass                     2
Name: count, dtype: int64

In [558]:
914+74+27+2

1017

Weird. There were 1019 "Save attempts" that are associated with the "Goal" or "own_goal" tag. 

There are 1017 other events (Shot, Free Kick, Others on the ball, Pass) that are associated with the "Goal" or "own_goal" tag.

However, by researching that season, I know that there were 1018 goals. So both are just 1 away from the correct result. I will locate the discrepency and try and explain it.

In [559]:
Save_attempt_goals=events[
    events['Label'].apply(lambda x: ('Goal' in x) | ('own_goal' in x))
    &
    (events['eventName'] == "Save attempt")
    ]

In [560]:
Other_goals=events[
    events['Label'].apply(lambda x: ('Goal' in x) | ('own_goal' in x))
    &
    (events['eventName'] != "Save attempt")
    ]

In [561]:
setA=set(Save_attempt_goals.groupby(['matchId'])['Label'].count().reset_index()['matchId'])

In [562]:
setB=set(Other_goals.groupby(['matchId'])['Label'].count().reset_index()['matchId'])

In [563]:
setA-setB

{2499781}

In [564]:
weirdmatch=events[events['matchId']==2499781]

In [565]:
weirdmatch[weirdmatch['Label'].apply(lambda x: ('Goal' in x) | ('own_goal' in x))]['eventName'].value_counts()

eventName
Save attempt    1
Name: count, dtype: int64

In [566]:
matches[matches['wyId']==2499781]

,gameweek,dateutc,winner,wyId,label,home.side,home.teamName,home.score,home.scoreHT,away.side,away.teamName,away.score,away.scoreHT,home.numShots,away.numShots
66,7,2017-09-30 16:30:00,Manchester City,2499781,"Chelsea - Manchester City, 0 - 1",home,Chelsea,0,0,away,Manchester City,1,0,4,14


In [567]:
Save_attempt_goals

,subEventName,playerId,matchId,eventName,teamName,matchPeriod,eventSec,pos_orig_y,pos_orig_x,pos_dest_y,pos_dest_x,Label
47,Reflexes,8480,2499719,Save attempt,Leicester City,1H,96.970614,100,100,59,12,"[Goal, glb, not accurate]"
92,Reflexes,7882,2499719,Save attempt,Arsenal,1H,256.547834,0,0,48,4,"[Goal, gt, not accurate]"
555,Reflexes,7882,2499719,Save attempt,Arsenal,1H,1712.908250,0,0,46,6,"[Goal, gc, not accurate]"
899,Reflexes,8480,2499719,Save attempt,Leicester City,1H,2817.604704,100,100,50,8,"[Goal, gc, not accurate]"
1108,Reflexes,7882,2499719,Save attempt,Arsenal,2H,636.247679,0,0,46,8,"[Goal, glb, not accurate]"
...,...,...,...,...,...,...,...,...,...,...,...,...
641383,Reflexes,93084,2500097,Save attempt,Leicester City,2H,1839.090016,0,71,0,71,"[Goal, gr, not accurate]"
642259,Reflexes,10131,2500098,Save attempt,Everton,1H,2289.169397,17,57,27,43,"[Goal, gbr, not accurate]"
642749,Save attempt,10131,2500098,Save attempt,Everton,2H,1076.658523,16,58,13,65,"[Goal, gc, not accurate]"
642889,Reflexes,3582,2500098,Save attempt,West Ham United,2H,1739.579587,61,62,74,54,"[Goal, gr, not accurate]"


In [568]:
table1=Save_attempt_goals.groupby(['matchId'])['Label'].count().reset_index()
table1

,matchId,Label
0,2499719,7
1,2499720,2
2,2499721,5
3,2499722,3
4,2499723,1
...,...,...
343,2500094,3
344,2500095,1
345,2500096,3
346,2500097,9


In [569]:
non_nil_matches=matches[~((matches['home.score']==0)
        & (matches['away.score']==0))
]




In [570]:
non_nil_matches=non_nil_matches.copy()

non_nil_matches['total_match_goals']=non_nil_matches['home.score']+non_nil_matches['away.score']

In [571]:
table2=non_nil_matches.groupby(['wyId'])['total_match_goals'].sum().reset_index()
table2

,wyId,total_match_goals
0,2499719,7
1,2499720,2
2,2499721,5
3,2499722,3
4,2499723,1
...,...,...
343,2500094,3
344,2500095,1
345,2500096,3
346,2500097,9


In [572]:
# Merge the tables on matchId and wyId columns
merged_df = table1.merge(table2, left_on='matchId', right_on='wyId', how='inner', suffixes=('_label', '_goals'))

# Filter rows where Label and total_match_goals are different
mismatched_rows = merged_df[merged_df['Label'] != merged_df['total_match_goals']]

# Display mismatched rows
mismatched_rows[['matchId', 'Label', 'total_match_goals']]

,matchId,Label,total_match_goals
246,2499988,3,2


In [573]:
weirdmatch2=events[events['matchId']==2499988]

In [574]:
weirdmatch2[weirdmatch2['Label'].apply(lambda x: ('Goal' in x) | ('own_goal' in x))][['eventName','teamName']].value_counts()

eventName     teamName       
Save attempt  Watford            3
Shot          West Ham United    2
Name: count, dtype: int64

In [575]:
matches[matches['wyId']==2499988]

,gameweek,dateutc,winner,wyId,label,home.side,home.teamName,home.score,home.scoreHT,away.side,away.teamName,away.score,away.scoreHT,home.numShots,away.numShots
263,27,2018-02-10 15:00:00,West Ham United,2499988,"West Ham United - Watford, 2 - 0",home,West Ham United,2,1,away,Watford,0,0,7,8


##### grouping by match and home team

In [576]:
numHomeGoals=matches.groupby(['dateutc','wyId','home.teamName'])['home.score'].first().reset_index()
numHomeGoals.rename(columns={'home.teamName':'teamName','home.score':'NumGoals'}, inplace=True)

##### grouping by match and home team

In [577]:
numAwayGoals=matches.groupby(['dateutc','wyId','away.teamName'])['away.score'].first().reset_index()
numAwayGoals.rename(columns={'away.teamName':'teamName','away.score':'NumGoals'}, inplace=True)


In [578]:
allgoals=pd.concat([numHomeGoals,numAwayGoals],axis=0).sort_values(by="dateutc")

In [579]:
# Calculate cumulative mean excluding the current row's goals
allgoals['GoalsPerGame'] = allgoals.groupby('teamName')['NumGoals'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count()
)


In [580]:
# Calculate rolling mean over the last 5 rows excluding the current row's goals
allgoals['GoalsPerGame_last5'] = allgoals.groupby('teamName')['NumGoals'].transform(
    lambda x: x.shift(1).rolling(window=5, min_periods=1).mean()
)

In [581]:
matches=matches.merge(
    allgoals, 
    how='left', 
    left_on=['wyId','home.teamName'], 
    right_on=['wyId','teamName'],
    suffixes=('','_y')
).rename(columns={'GoalsPerGame':'home.GoalsPerGame', 'GoalsPerGame_last5':'home.GoalsPerGame_last5'})

matches.drop(['dateutc_y','teamName','NumGoals'], axis=1, inplace=True)

In [582]:
matches=matches.merge(
    allgoals, 
    how='left', 
    left_on=['wyId','away.teamName'], 
    right_on=['wyId','teamName'],
    suffixes=('','_y')
).rename(columns={'GoalsPerGame':'away.GoalsPerGame', 'GoalsPerGame_last5':'away.GoalsPerGame_last5'})

matches.drop(['dateutc_y','teamName','NumGoals'], axis=1, inplace=True)

In [583]:
matches.columns

Index(['gameweek', 'dateutc', 'winner', 'wyId', 'label', 'home.side',
       'home.teamName', 'home.score', 'home.scoreHT', 'away.side',
       'away.teamName', 'away.score', 'away.scoreHT', 'home.numShots',
       'away.numShots', 'home.GoalsPerGame', 'home.GoalsPerGame_last5',
       'away.GoalsPerGame', 'away.GoalsPerGame_last5'],
      dtype='object')

### Conceded

I also want to get the average goals conceded to date and the average goals conceded in the last 5 matches

In [584]:
numHomeConceded=matches.groupby(['dateutc','wyId','home.teamName'])['away.score'].first().reset_index()
numHomeConceded.rename(columns={'home.teamName':'teamName','away.score':'NumConceded'}, inplace=True)

In [585]:
numAwayConceded=matches.groupby(['dateutc','wyId','away.teamName'])['home.score'].first().reset_index()
numAwayConceded.rename(columns={'away.teamName':'teamName','home.score':'NumConceded'}, inplace=True)

In [586]:
allConceded=pd.concat([numHomeConceded,numAwayConceded],axis=0).sort_values(by="dateutc")

In [587]:
# Calculate cumulative mean excluding the current row's goals
allConceded['ConcededPerGame'] = allConceded.groupby('teamName')['NumConceded'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count()
)


In [588]:
# Calculate rolling mean over the last 5 rows excluding the current row's goals
allConceded['ConcededPerGame_last5'] = allConceded.groupby('teamName')['NumConceded'].transform(
    lambda x: x.shift(1).rolling(window=5, min_periods=1).mean()
)

In [589]:
allConceded

,dateutc,wyId,teamName,NumConceded,ConcededPerGame,ConcededPerGame_last5
0,2017-08-11 18:45:00,2499719,Arsenal,3,NaN,NaN
0,2017-08-11 18:45:00,2499719,Leicester City,4,NaN,NaN
1,2017-08-12 11:30:00,2499727,Watford,3,NaN,NaN
1,2017-08-12 11:30:00,2499727,Liverpool,3,NaN,NaN
2,2017-08-12 14:00:00,2499721,Chelsea,3,NaN,NaN
...,...,...,...,...,...,...
372,2018-05-13 14:00:00,2500091,Huddersfield Town,1,1.540541,0.8
371,2018-05-13 14:00:00,2500090,Crystal Palace,0,1.486486,1.0
370,2018-05-13 14:00:00,2500089,Burnley,2,1.000000,1.8
378,2018-05-13 14:00:00,2500097,Leicester City,5,1.486486,2.0


In [590]:
matches=matches.merge(
    allConceded, 
    how='left', 
    left_on=['wyId','home.teamName'], 
    right_on=['wyId','teamName'],
    suffixes=('','_y')
).rename(columns={'ConcededPerGame':'home.ConcededPerGame', 'ConcededPerGame_last5':'home.ConcededPerGame_last5'})

matches.drop(['dateutc_y','teamName','NumConceded'], axis=1, inplace=True)

In [591]:
matches=matches.merge(
    allConceded, 
    how='left', 
    left_on=['wyId','away.teamName'], 
    right_on=['wyId','teamName'],
    suffixes=('','_y')
).rename(columns={'ConcededPerGame':'away.ConcededPerGame', 'ConcededPerGame_last5':'away.ConcededPerGame_last5'})

matches.drop(['dateutc_y','teamName','NumConceded'], axis=1, inplace=True)

In [592]:
matches.columns

Index(['gameweek', 'dateutc', 'winner', 'wyId', 'label', 'home.side',
       'home.teamName', 'home.score', 'home.scoreHT', 'away.side',
       'away.teamName', 'away.score', 'away.scoreHT', 'home.numShots',
       'away.numShots', 'home.GoalsPerGame', 'home.GoalsPerGame_last5',
       'away.GoalsPerGame', 'away.GoalsPerGame_last5', 'home.ConcededPerGame',
       'home.ConcededPerGame_last5', 'away.ConcededPerGame',
       'away.ConcededPerGame_last5'],
      dtype='object')

In [595]:
temp=matches[['gameweek','winner','label','home.teamName','away.teamName', 'home.GoalsPerGame', 'home.GoalsPerGame_last5',
       'away.GoalsPerGame', 'away.GoalsPerGame_last5','home.ConcededPerGame',
       'home.ConcededPerGame_last5', 'away.ConcededPerGame',
       'away.ConcededPerGame_last5']]

In [597]:
temp[(temp['home.teamName']=='Arsenal')| (temp['away.teamName']=='Arsenal')].head(10)

,gameweek,winner,label,home.teamName,away.teamName,home.GoalsPerGame,home.GoalsPerGame_last5,away.GoalsPerGame,away.GoalsPerGame_last5,home.ConcededPerGame,home.ConcededPerGame_last5,away.ConcededPerGame,away.ConcededPerGame_last5
0,1,Arsenal,"Arsenal - Leicester City, 4 - 3",Arsenal,Leicester City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2,Stoke City,"Stoke City - Arsenal, 1 - 0",Stoke City,Arsenal,0.000000,0.000000,4.000000,4.000000,1.000000,1.000000,3.000000,3.000000
28,3,Liverpool,"Liverpool - Arsenal, 4 - 0",Liverpool,Arsenal,2.000000,2.000000,2.000000,2.000000,1.500000,1.500000,2.000000,2.000000
35,4,Arsenal,"Arsenal - AFC Bournemouth, 3 - 0",Arsenal,AFC Bournemouth,1.333333,1.333333,0.333333,0.333333,2.666667,2.666667,1.666667,1.666667
48,5,NaN,"Chelsea - Arsenal, 0 - 0",Chelsea,Arsenal,2.000000,2.000000,1.750000,1.750000,1.250000,1.250000,2.000000,2.000000
59,6,Arsenal,"Arsenal - West Bromwich Albion, 2 - 0",Arsenal,West Bromwich Albion,1.400000,1.400000,0.800000,0.800000,1.600000,1.600000,0.800000,0.800000
67,7,Arsenal,"Arsenal - Brighton & Hove Albion, 2 - 0",Arsenal,Brighton & Hove Albion,1.500000,1.000000,0.833333,1.000000,1.333333,1.000000,1.166667,1.000000
76,8,Watford,"Watford - Arsenal, 2 - 1",Watford,Arsenal,1.571429,1.200000,1.571429,1.400000,1.714286,1.800000,1.142857,0.800000
88,9,Arsenal,"Everton - Arsenal, 2 - 5",Everton,Arsenal,0.625000,0.600000,1.500000,1.600000,1.625000,2.000000,1.250000,0.400000
92,10,Arsenal,"Arsenal - Swansea City, 2 - 1",Arsenal,Swansea City,1.888889,2.000000,0.666667,0.800000,1.333333,0.800000,1.111111,1.000000
